# Tutorial 1 - RNN Sequence classifier

In this notebook, we will predict the winner of a basketball game based on the scores observed in the first 3 quarters of the game. Each column represents the beginning of a minute during the game. (There are 12 minutes in each quarter. There are  3 quarters in the data, so we have 36 columns as input variables in chronological order.) The values captured in each column represent the score difference observed at that minute (home score minus away score)<br><br>

The last column `W` represents whether the home team (1) or the away team (0) won the game. This is the target variable. <br><br>
**Our unit of analysis is a single game.**

In [1]:
# Common imports
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

np.random.seed(1)
tf.random.set_seed(1)

## Get the data

In [2]:
data = pd.read_csv("./data/basketball.csv")

In [3]:
data.shape

(1230, 37)

In [4]:
data.head()

,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,...,M28,M29,M30,M31,M32,M33,M34,M35,M36,W
0,-2,-1,1,1,-1,-3,-1,0,3,6,...,9,11,10,7,7,4,6,2,1,1
1,0,2,7,6,10,8,8,6,0,6,...,7,11,11,14,14,15,13,13,13,0
2,0,-2,2,0,5,4,5,3,5,3,...,9,11,13,13,12,17,15,15,12,1
3,0,2,0,3,4,3,5,4,3,1,...,10,6,7,7,8,8,8,8,8,1
4,0,-2,-2,0,3,-2,-7,-5,-7,-4,...,10,10,15,13,11,11,11,13,10,1


In [5]:
y = data['W']
X = data.drop('W', axis=1)

## Split the data

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Data Transformation

In [7]:
#Target variables need to be an array with integer type
y_train = np.array(y_train)
y_test = np.array(y_test)

y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [8]:
#Check the first 10 values of the train_y data set
y_train[0:10]

array([0, 1, 1, 1, 1, 1, 0, 0, 0, 0])

In [9]:
#Convert input variables to a 2-D array with float data type
X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [10]:
X_train

array([[ -3.,  -3.,  -2., ...,  -5.,  -3.,  -7.],
       [  0.,   1.,  -1., ...,  10.,  12.,  15.],
       [ -3.,  -2.,  -2., ...,  14.,  12.,  12.],
       ...,
       [  2.,   6.,   8., ...,  -2.,   1.,   0.],
       [ -2.,   1.,  -1., ..., -19., -21., -14.],
       [  2.,   1.,   3., ...,   0.,   2.,   4.]], dtype=float32)

In [11]:
#Keras expects a different input format:
#Data needs to have 3 dimensions

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [12]:
X_train.shape, y_train.shape

((861, 36, 1), (861,))

In [13]:
X_train

array([[[ -3.],
        [ -3.],
        [ -2.],
        ...,
        [ -5.],
        [ -3.],
        [ -7.]],

       [[  0.],
        [  1.],
        [ -1.],
        ...,
        [ 10.],
        [ 12.],
        [ 15.]],

       [[ -3.],
        [ -2.],
        [ -2.],
        ...,
        [ 14.],
        [ 12.],
        [ 12.]],

       ...,

       [[  2.],
        [  6.],
        [  8.],
        ...,
        [ -2.],
        [  1.],
        [  0.]],

       [[ -2.],
        [  1.],
        [ -1.],
        ...,
        [-19.],
        [-21.],
        [-14.]],

       [[  2.],
        [  1.],
        [  3.],
        ...,
        [  0.],
        [  2.],
        [  4.]]], dtype=float32)

# A normal (cross-sectional) NN

This model assumes that the data is NOT a time-series data set. It treats the data as cross-sectional and the columns being independent of each other.

In [14]:
model = keras.models.Sequential([
    
    keras.layers.Flatten(input_shape=[36, 1]),
    keras.layers.Dense(36, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
    
])

In [15]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])


history = model.fit(X_train, y_train, epochs=50,
                    validation_data=(X_test, y_test))

Epoch 1/50
27/27 [==============================] - 2s 14ms/step - loss: 0.9744 - accuracy: 0.7213 - val_loss: 0.5195 - val_accuracy: 0.7940
Epoch 2/50
27/27 [==============================] - 0s 5ms/step - loss: 0.5188 - accuracy: 0.7909 - val_loss: 0.5293 - val_accuracy: 0.7615
Epoch 3/50
27/27 [==============================] - 0s 5ms/step - loss: 0.4490 - accuracy: 0.8026 - val_loss: 0.5196 - val_accuracy: 0.8049
Epoch 4/50
27/27 [==============================] - 0s 5ms/step - loss: 0.4124 - accuracy: 0.8258 - val_loss: 0.5806 - val_accuracy: 0.7507
Epoch 5/50
27/27 [==============================] - 0s 5ms/step - loss: 0.4022 - accuracy: 0.8200 - val_loss: 0.5220 - val_accuracy: 0.7534
Epoch 6/50
27/27 [==============================] - 0s 5ms/step - loss: 0.3950 - accuracy: 0.8304 - val_loss: 0.5616 - val_accuracy: 0.7371
Epoch 7/50
27/27 [==============================] - 0s 5ms/step - loss: 0.3595 - accuracy: 0.8374 - val_loss: 0.5000 - val_accuracy: 0.8049
Epoch 8/50
27/27 [=

In [16]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.7520841956138611, 0.7533875107765198]

In [17]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.75
accuracy: 75.34%


# Simple RNN with one layer

In [18]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.SimpleRNN(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [19]:
from tensorflow.keras.callbacks import EarlyStopping

optimizer = tf.keras.optimizers.Nadam(learning_rate=0.01)

# If multiclass, use "sparse_categorical_crossentropy" as the loss function
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')

history = model.fit(
    X_train, 
    y_train, 
    epochs=50,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop]
)

Epoch 1/50
27/27 [==============================] - 3s 23ms/step - loss: 0.4378 - accuracy: 0.8060 - val_loss: 0.3850 - val_accuracy: 0.8347
Epoch 2/50
27/27 [==============================] - 0s 11ms/step - loss: 0.4176 - accuracy: 0.8026 - val_loss: 0.4061 - val_accuracy: 0.8049
Epoch 3/50
27/27 [==============================] - 0s 11ms/step - loss: 0.4136 - accuracy: 0.7979 - val_loss: 0.4043 - val_accuracy: 0.8076
Epoch 4/50
27/27 [==============================] - 0s 10ms/step - loss: 0.4071 - accuracy: 0.8026 - val_loss: 0.3940 - val_accuracy: 0.8211
Epoch 5/50
27/27 [==============================] - 0s 11ms/step - loss: 0.3944 - accuracy: 0.8211 - val_loss: 0.4052 - val_accuracy: 0.8103
Epoch 6/50
27/27 [==============================] - 0s 11ms/step - loss: 0.3952 - accuracy: 0.8130 - val_loss: 0.4045 - val_accuracy: 0.8103
Epoch 6: early stopping


In [20]:
# evaluate the model
scores = model.evaluate(X_test, y_test, verbose=0)

scores

# In results, first is loss, second is accuracy

[0.4044952392578125, 0.8102980852127075]

In [21]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.40
accuracy: 81.03


In [22]:
# Predictions are probabilities.

predictions = model.predict(X_test)

12/12 [==============================] - 0s 4ms/step


In [23]:
# Rounding the probabilities determines 1 or 0

np.round(predictions)[:10] # show first 10 predictions

array([[0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.]], dtype=float32)

In [24]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, np.round(predictions))

array([[124,  43],
       [ 27, 175]], dtype=int64)

# Deep RNN

**Be careful: when stacking RNN layers, you have to set "return_sequences" to True. This enables the layer to send a "sequence" of values to the next layer -- jut like how it uses a sequence of values for training.**

**Since the last layer is DENSE, it can't take sequence data. Therefore, you CANNOT return sequences from the previous layer. So, remove** `return_sequences` **from previous layer.**

In [25]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[n_steps, n_inputs] ),
    keras.layers.SimpleRNN(32, return_sequences=True),
    keras.layers.SimpleRNN(32), 
    keras.layers.Dense(1, activation='sigmoid')
])

In [26]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data = (X_test, y_test), 
    callbacks=[early_stop])  # we defined this early_stop callback function earlier

Epoch 1/20
27/27 [==============================] - 6s 39ms/step - loss: 0.4567 - accuracy: 0.7747 - val_loss: 0.3923 - val_accuracy: 0.8130
Epoch 2/20
27/27 [==============================] - 1s 20ms/step - loss: 0.4242 - accuracy: 0.7921 - val_loss: 0.4332 - val_accuracy: 0.7886
Epoch 3/20
27/27 [==============================] - 1s 21ms/step - loss: 0.4200 - accuracy: 0.8060 - val_loss: 0.4110 - val_accuracy: 0.8076
Epoch 4/20
27/27 [==============================] - 1s 20ms/step - loss: 0.4087 - accuracy: 0.7933 - val_loss: 0.3965 - val_accuracy: 0.8347
Epoch 5/20
27/27 [==============================] - 1s 19ms/step - loss: 0.4155 - accuracy: 0.7991 - val_loss: 0.3895 - val_accuracy: 0.8184
Epoch 6/20
27/27 [==============================] - 1s 20ms/step - loss: 0.4033 - accuracy: 0.8049 - val_loss: 0.4049 - val_accuracy: 0.8076
Epoch 7/20
27/27 [==============================] - 1s 20ms/step - loss: 0.3864 - accuracy: 0.8002 - val_loss: 0.4091 - val_accuracy: 0.8076
Epoch 8/20
27

In [27]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)

scores
# In results, first is loss, second is accuracy

[0.4326169788837433, 0.7859078645706177]

In [28]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.43
accuracy: 78.59


# LSTM with one layer

In [29]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [30]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test),
    callbacks=[early_stop]) # we defined this early_stop callback function earlier

Epoch 1/20
27/27 [==============================] - 4s 46ms/step - loss: 0.5143 - accuracy: 0.7213 - val_loss: 0.3972 - val_accuracy: 0.8266
Epoch 2/20
27/27 [==============================] - 0s 17ms/step - loss: 0.4233 - accuracy: 0.7991 - val_loss: 0.4113 - val_accuracy: 0.8103
Epoch 3/20
27/27 [==============================] - 0s 17ms/step - loss: 0.4159 - accuracy: 0.8002 - val_loss: 0.3943 - val_accuracy: 0.8266
Epoch 4/20
27/27 [==============================] - 0s 18ms/step - loss: 0.4102 - accuracy: 0.8014 - val_loss: 0.3958 - val_accuracy: 0.8184
Epoch 5/20
27/27 [==============================] - 0s 17ms/step - loss: 0.4072 - accuracy: 0.8084 - val_loss: 0.4025 - val_accuracy: 0.8184
Epoch 6/20
27/27 [==============================] - 0s 17ms/step - loss: 0.4008 - accuracy: 0.8014 - val_loss: 0.4090 - val_accuracy: 0.7995
Epoch 7/20
27/27 [==============================] - 0s 18ms/step - loss: 0.3975 - accuracy: 0.8002 - val_loss: 0.4077 - val_accuracy: 0.8211
Epoch 8/20
27

In [31]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.41734516620635986, 0.8021680116653442]

In [32]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.42
accuracy: 80.22


# LSTM with more layers

In [33]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.LSTM(32, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid')
])

In [34]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 9s 92ms/step - loss: 0.5033 - accuracy: 0.7573 - val_loss: 0.4089 - val_accuracy: 0.8293
Epoch 2/20
27/27 [==============================] - 1s 45ms/step - loss: 0.4312 - accuracy: 0.7909 - val_loss: 0.4096 - val_accuracy: 0.8022
Epoch 3/20
27/27 [==============================] - 1s 44ms/step - loss: 0.4180 - accuracy: 0.7933 - val_loss: 0.4151 - val_accuracy: 0.7967
Epoch 4/20
27/27 [==============================] - 1s 44ms/step - loss: 0.4120 - accuracy: 0.8014 - val_loss: 0.3962 - val_accuracy: 0.8184
Epoch 5/20
27/27 [==============================] - 1s 44ms/step - loss: 0.4136 - accuracy: 0.8026 - val_loss: 0.4150 - val_accuracy: 0.7967
Epoch 6/20
27/27 [==============================] - 1s 42ms/step - loss: 0.4093 - accuracy: 0.7991 - val_loss: 0.4086 - val_accuracy: 0.8266
Epoch 7/20
27/27 [==============================] - 1s 43ms/step - loss: 0.3991 - accuracy: 0.7967 - val_loss: 0.4361 - val_accuracy: 0.7886
Epoch 8/20
27

In [35]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.4647825360298157, 0.7859078645706177]

In [36]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.46
accuracy: 78.59%


# GRU with one layer

In [37]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(32, input_shape=[n_steps, n_inputs]),
    keras.layers.Dense(1, activation='sigmoid')
])

In [38]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test),
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 4s 32ms/step - loss: 0.4565 - accuracy: 0.7724 - val_loss: 0.3901 - val_accuracy: 0.8347
Epoch 2/20
27/27 [==============================] - 0s 15ms/step - loss: 0.4129 - accuracy: 0.7944 - val_loss: 0.4025 - val_accuracy: 0.8130
Epoch 3/20
27/27 [==============================] - 0s 16ms/step - loss: 0.4171 - accuracy: 0.7991 - val_loss: 0.3914 - val_accuracy: 0.8293
Epoch 4/20
27/27 [==============================] - 0s 16ms/step - loss: 0.4099 - accuracy: 0.7875 - val_loss: 0.3920 - val_accuracy: 0.8293
Epoch 5/20
27/27 [==============================] - 0s 15ms/step - loss: 0.4061 - accuracy: 0.8014 - val_loss: 0.3957 - val_accuracy: 0.8293
Epoch 6/20
27/27 [==============================] - 0s 15ms/step - loss: 0.4033 - accuracy: 0.7956 - val_loss: 0.4063 - val_accuracy: 0.8238
Epoch 6: early stopping


In [39]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.4062686860561371, 0.8238482475280762]

In [40]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.41
accuracy: 82.38


# GRU with more layers

In [41]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.GRU(32, return_sequences=True, input_shape=[n_steps, n_inputs]),
    keras.layers.GRU(32, return_sequences=True),
    keras.layers.GRU(32),
    keras.layers.Dense(1, activation='sigmoid')
])

In [42]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 9s 86ms/step - loss: 0.4531 - accuracy: 0.7933 - val_loss: 0.3958 - val_accuracy: 0.8157
Epoch 2/20
27/27 [==============================] - 1s 42ms/step - loss: 0.4203 - accuracy: 0.7933 - val_loss: 0.4049 - val_accuracy: 0.8211
Epoch 3/20
27/27 [==============================] - 1s 42ms/step - loss: 0.4191 - accuracy: 0.8002 - val_loss: 0.4076 - val_accuracy: 0.8103
Epoch 4/20
27/27 [==============================] - 1s 44ms/step - loss: 0.4120 - accuracy: 0.7967 - val_loss: 0.3926 - val_accuracy: 0.8238
Epoch 5/20
27/27 [==============================] - 1s 44ms/step - loss: 0.4135 - accuracy: 0.8002 - val_loss: 0.4120 - val_accuracy: 0.8103
Epoch 6/20
27/27 [==============================] - 1s 43ms/step - loss: 0.4124 - accuracy: 0.7956 - val_loss: 0.4142 - val_accuracy: 0.8157
Epoch 7/20
27/27 [==============================] - 1s 45ms/step - loss: 0.4039 - accuracy: 0.8026 - val_loss: 0.4113 - val_accuracy: 0.8157
Epoch 8/20
27

In [43]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.41735270619392395, 0.8157181739807129]

In [44]:
# extract the accuracy from model.evaluate

print("%s: %.2f" % (model.metrics_names[0], scores[0]))
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


loss: 0.42
accuracy: 81.57%


# Conv1D

In [45]:
n_steps = 36
n_inputs = 1

model = keras.models.Sequential([
    keras.layers.Conv1D(filters=10, kernel_size=3, strides=2, padding="valid", input_shape=[n_steps, n_inputs]),
    keras.layers.Conv1D(filters=20, kernel_size=3, strides=1, padding="valid", dilation_rate=2),
    keras.layers.LSTM(32, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 17, 10)            40        
                                                                 
 conv1d_1 (Conv1D)           (None, 13, 20)            620       
                                                                 
 lstm_4 (LSTM)               (None, 13, 32)            6784      
                                                                 
 lstm_5 (LSTM)               (None, 32)                8320      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 15,797
Trainable params: 15,797
Non-trainable params: 0
_________________________________________________________________


In [46]:
np.random.seed(42)
tf.random.set_seed(42)

optimizer = keras.optimizers.Nadam(learning_rate=0.01)

model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['accuracy'])

history = model.fit(
    X_train, 
    y_train, 
    epochs=20,
    validation_data=(X_test, y_test), 
    callbacks=[early_stop] # we defined this early_stop callback function earlier
)

Epoch 1/20
27/27 [==============================] - 7s 49ms/step - loss: 0.5265 - accuracy: 0.7224 - val_loss: 0.4283 - val_accuracy: 0.8157
Epoch 2/20
27/27 [==============================] - 0s 16ms/step - loss: 0.4555 - accuracy: 0.7851 - val_loss: 0.4235 - val_accuracy: 0.8130
Epoch 3/20
27/27 [==============================] - 0s 16ms/step - loss: 0.4361 - accuracy: 0.7816 - val_loss: 0.4106 - val_accuracy: 0.7995
Epoch 4/20
27/27 [==============================] - 0s 17ms/step - loss: 0.4225 - accuracy: 0.8072 - val_loss: 0.4182 - val_accuracy: 0.8103
Epoch 5/20
27/27 [==============================] - 0s 17ms/step - loss: 0.4260 - accuracy: 0.7967 - val_loss: 0.4371 - val_accuracy: 0.7940
Epoch 6/20
27/27 [==============================] - 0s 18ms/step - loss: 0.4147 - accuracy: 0.7933 - val_loss: 0.4514 - val_accuracy: 0.7805
Epoch 7/20
27/27 [==============================] - 0s 16ms/step - loss: 0.3982 - accuracy: 0.8165 - val_loss: 0.4272 - val_accuracy: 0.7886
Epoch 8/20
27

In [47]:
# evaluate the model

scores = model.evaluate(X_test, y_test, verbose=0)
scores

# In results, first is loss, second is accuracy

[0.473165363073349, 0.7669376730918884]

In [48]:
# extract the accuracy from model.evaluate
print(f"{model.metrics_names[0]:s}: {scores[0]:.2f}")
print(f"{model.metrics_names[1]:s}: {scores[1]*100:.2f}")

loss: 0.47
accuracy: 76.69
